In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, regexp_replace

spark = SparkSession \
    .builder \
    .appName("WordCount") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/07/30 23:11:32 WARN Utils: Your hostname, Pramod resolves to a loopback address: 127.0.1.1; using 172.23.91.132 instead (on interface eth0)
23/07/30 23:11:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/30 23:11:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Create Data Producing NetCat Server: 
```
nc -lk 9999
```

In [2]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

words = words.withColumn("word", regexp_replace("word", "[^0-9a-zA-Z_\-]+", ""))

# Generate running word count
wordCounts = words.groupBy("word").count()

23/07/30 23:11:53 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [3]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

23/07/30 23:12:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ffdf1785-5d86-498b-812f-9f39ad416ce1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/07/30 23:12:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    2|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|  you|    1|
|  how|    1|
|hello|    2|
|  are|    1|
+-----+-----+



I/P given to NetCat server:
```
(base) root@Pramod:~# nc -lk 9999
hello
hello
how are you?
```